# U-Net for Image Segmentation: A Complete Educational Guide

## Introduction

This notebook provides a comprehensive educational guide to implementing U-Net architecture for semantic image segmentation using the Carvana Image Masking Challenge dataset. U-Net is a convolutional neural network architecture specifically designed for biomedical image segmentation, but it has proven effective for many other segmentation tasks.

### What You Will Learn

1. Understanding U-Net architecture and its components
2. Data preprocessing techniques for image segmentation
3. Implementation of encoder-decoder networks with skip connections
4. Training strategies for segmentation models
5. Evaluation metrics for segmentation tasks
6. Practical application on real-world dataset

### Dataset Overview

The Carvana Image Masking Challenge dataset contains high-resolution images of cars photographed from different angles, along with their corresponding binary masks that segment the car from the background. This dataset is perfect for learning image segmentation techniques.

### Prerequisites

- Basic understanding of deep learning concepts
- Familiarity with convolutional neural networks
- Python programming knowledge
- Understanding of image processing fundamentals

## 1. Import Required Libraries

### Step-by-Step Instructions:

1. **Deep Learning Framework**: Import TensorFlow and Keras for building and training the neural network
   - Import tensorflow as tf
   - Import specific Keras modules: layers, models, optimizers, callbacks
   - Set random seeds for reproducibility

2. **Data Manipulation**: Import libraries for handling arrays and data structures
   - Import NumPy for numerical operations
   - Import Pandas for data manipulation and CSV handling
   - Import os and glob for file system operations

3. **Image Processing**: Import libraries for image operations
   - Import OpenCV (cv2) for image preprocessing
   - Import PIL (Python Imaging Library) for image loading and manipulation
   - Import skimage for additional image processing functions

4. **Visualization**: Import plotting libraries
   - Import Matplotlib for creating plots and visualizations
   - Import seaborn for enhanced statistical visualizations
   - Set up matplotlib inline for Jupyter notebook display

5. **Utility Libraries**: Import additional helpful libraries
   - Import tqdm for progress bars during training
   - Import warnings to suppress unnecessary warnings
   - Import json for configuration file handling

### Why These Libraries?

- **TensorFlow/Keras**: Primary framework for building the U-Net model
- **NumPy**: Essential for array operations and mathematical computations
- **OpenCV**: Efficient image processing and computer vision operations
- **Matplotlib**: Visualizing training progress and results
- **tqdm**: Monitoring training and data loading progress

In [ ]:
import random, os, glob, json, tqdm
import tensorflow as tf

from keras import layers, models, optimizers, callbacks
from keras.layers import Input,  concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, BatchNormalization, Dropout
from keras.models import Model

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image, ImageEnhance
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2, skimage

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

%matplotlib inline

## 2. Download and Extract Carvana Dataset

1. **Download the Dataset**:
   - Use Kaggle API to download the Carvana Image Masking Challenge dataset
   - Command: `kaggle competitions download -c carvana-image-masking-challenge`
   - This will download all competition files including:
     - Train images (train.zip)
     - Train masks (train_masks.zip)
     - Test images (test.zip)
     - Sample submission file
     - Metadata

2. **Extract the Files**:
   - Create organized directory structure for the project
   - Extract train.zip to train/ folder
   - Extract train_masks.zip to train_masks/ folder
   - Extract test.zip to test/ folder
   - Keep the original zip files for backup

5. **Verify Dataset Structure**:
   - Check that all folders contain expected files
   - Verify image and mask file naming conventions
   - Count total number of training images and masks
   - Ensure masks correspond to training images

### Expected Directory Structure:
```
carvana-dataset/
├── train/                 # Training images
├── train_masks/          # Training masks (binary images)
├── test/                 # Test images
├── sample_submission.csv # Sample submission format
└── metadata.csv         # Additional dataset information
```

### Important Notes:
- Dataset size is approximately 5GB
- Training set contains 5,088 images
- Each car is photographed from 16 different angles
- Masks are binary images (car = white, background = black)

In [ ]:
# # 2. Download and Extract Carvana Dataset

# # Download the dataset using Kaggle API
# # !kaggle competitions download -c carvana-image-masking-challenge -p ./carvana-dataset

# # Extract the files
# # import zipfile
# # with zipfile.ZipFile('./carvana-dataset/train.zip', 'r') as zip_ref:
# #     zip_ref.extractall('./carvana-dataset/train/')
# # with zipfile.ZipFile('./carvana-dataset/train_masks.zip', 'r') as zip_ref:
# #     zip_ref.extractall('./carvana-dataset/train_masks/')
# # with zipfile.ZipFile('./carvana-dataset/test.zip', 'r') as zip_ref:
# #     zip_ref.extractall('./carvana-dataset/test/')

# # Verify dataset structure
# # import os
# # print("Train images:", len(os.listdir('./carvana-dataset/train/')))
# # print("Train masks:", len(os.listdir('./carvana-dataset/train_masks/')))
# # print("Test images:", len(os.listdir('./carvana-dataset/test/')))
# # print("Sample submission exists:", os.path.exists('./carvana-dataset/sample_submission.csv'))
# # print("Metadata exists:", os.path.exists('./carvana-dataset/metadata.csv'))

## 3. Data Exploration and Visualization

1. **Dataset Overview**:
   - Count total number of training images and masks
   - List first few image filenames to understand naming convention
   - Check if every training image has a corresponding mask

In [ ]:
# ---------------------------------------------------------------------------- #
#                                  DIRECTORIES                                 #
# ---------------------------------------------------------------------------- #

TRAIN_IMAGES_DIR = "carvana-dataset/train"
TRAIN_MASKS_DIR = "carvana-dataset/train_masks"
TEST_IMAGES_DIR = "carvana-dataset/test"

# ---------------------------------------------------------------------------- #
#                       LISTS TO STORE IMAGE DIRECTORIES                       #
# ---------------------------------------------------------------------------- #

train_set = []
train_mask_set = []
test_set = []

# ---------------------------------------------------------------------------- #
#                        PRINT COUNT OF IMAGES AND MASKS                       #
# ---------------------------------------------------------------------------- #

print("Train images: ", len(glob.glob(os.path.join(TRAIN_IMAGES_DIR, "*.*"))))
print("Train masks: ", len(glob.glob(os.path.join(TRAIN_MASKS_DIR, "*.*"))))
print("Test images: ", len(glob.glob(os.path.join(TEST_IMAGES_DIR, "*.*"))))

# ---------------------------------------------------------------------------- #
#                                APPENDING LISTS                               #
# ---------------------------------------------------------------------------- #

# TRAINING IMAGES
if os.path.exists(TRAIN_IMAGES_DIR):
    for image_filename in os.listdir(TRAIN_IMAGES_DIR):
        # Create full path to the image
        image_path = os.path.join(TRAIN_IMAGES_DIR, image_filename)
        train_set.append(image_path)
else:
    print(f"Directory {TRAIN_IMAGES_DIR} not found. Please check if the dataset is extracted correctly.")

# TRAINING MASKS
if os.path.exists(TRAIN_MASKS_DIR):
    for image_filename in os.listdir(TRAIN_MASKS_DIR):
        # Create full path to the image
        image_path = os.path.join(TRAIN_MASKS_DIR, image_filename)
        train_mask_set.append(image_path)
else:
    print(f"Directory {TRAIN_MASKS_DIR} not found. Please check if the dataset is extracted correctly.")

# TEST IMAGES
if os.path.exists(TEST_IMAGES_DIR):
    for image_filename in os.listdir(TEST_IMAGES_DIR):
        # Create full path to the image
        image_path = os.path.join(TEST_IMAGES_DIR, image_filename)
        test_set.append(image_path)
else:
    print(f"Directory {TEST_IMAGES_DIR} not found. Please check if the dataset is extracted correctly.")

# Print samples
print(train_set[:4])
print(train_mask_set[:4])
print(test_set[:4])

2. **Image Properties Analysis**:
   - Load several sample images and examine their properties:
     - Image dimensions (height, width, channels)
     - Color space (RGB, BGR)

In [ ]:
# ---------------------------------------------------------------------------- #
#                                LOADING IMAGES                                #
# ---------------------------------------------------------------------------- #

images_to_display = train_set[:3] + train_mask_set[:3] + test_set[:3]
images = [Image.open(path) for path in images_to_display]

fig, axes = plt.subplots(3, 3, figsize=(12, 4))
for ax, img in zip(axes.flat, images):
    ax.imshow(img)
    ax.axis('off')

plt.tight_layout()
plt.show()

# ---------------------------------------------------------------------------- #
#                               IMAGE PROPERTIES                               #
# ---------------------------------------------------------------------------- #

for img in images:
    print("Format: ", img.format)
    print("Size:", img.size)
    print("Mode:", img.mode)
    print("Info:", img.info)
    print("Palette:", img.palette)
    print("")


3. **Mask Properties Analysis**:
   - Load corresponding mask images and analyze:
     - Mask dimensions (should match original images)
     - Pixel value distribution (0 for background, 255 for car)
     - Data type and format
     - Verify binary nature of masks

In [ ]:
# ---------------------------------------------------------------------------- #
#                           MASK PROPERTIES ANALYSIS                           #
# ---------------------------------------------------------------------------- #

df_images_masks = pd.DataFrame({
    "image": train_set,
    "mask": train_mask_set
})
df_images_masks.head()

sample_masks = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in df_images_masks["mask"].head(3)]
for i, mask in enumerate(sample_masks):
    print(f"Mask {i+1}:")
    if mask is not None:
        print("Shape:", mask.shape)
        print("Data type:", mask.dtype)
        print("Unique pixel values:", np.unique(mask))
        print("Value counts:", dict(zip(*np.unique(mask, return_counts=True))))
    else:
        print("Mask could not be loaded. Check the file path or format.")
    print("")

4. **Visual Exploration**:
   - Display a grid of sample images with their corresponding masks
   - Show original image, mask, and masked image side by side
   - Create visualizations showing:
     - Different car types and colors
     - Various camera angles (16 per car)
     - Mask complexity variations

In [ ]:
# ---------------------------------------------------------------------------- #
#                 DISPLAY SAMPLES: ORIGINAL, MASK, MASKED IMAGE                #
# ---------------------------------------------------------------------------- #

fig, axes = plt.subplots(3, 5, figsize=(18, 8))

for idx in range(5):
    img_path = df_images_masks.iloc[idx]['image']
    mask_path = df_images_masks.iloc[idx]['mask']

    image = Image.open(img_path).convert('RGB')
    mask = Image.open(mask_path).convert('L')

    mask_np = np.array(mask)
    binary_mask_np = np.where(mask_np > 127, 255, 0).astype(np.uint8)
    binary_mask = Image.fromarray(binary_mask_np)

    background = Image.new('RGB', image.size, (0, 0, 0))
    masked_image = Image.composite(image, background, binary_mask)

    # Original image
    axes[0, idx].imshow(image)
    axes[0, idx].set_title(f"Image {idx+1}")
    axes[0, idx].axis('off')

    # Mask
    axes[1, idx].imshow(mask, cmap='gray')
    axes[1, idx].set_title("Mask")
    axes[1, idx].axis('off')

    # Masked image
    axes[2, idx].imshow(masked_image)
    axes[2, idx].set_title("Masked")
    axes[2, idx].axis('off')

plt.tight_layout()
plt.show()

5. **Statistical Analysis**:
   - Calculate mask coverage statistics:
     - Average percentage of car pixels per image
     - Distribution of car sizes in images
     - Aspect ratio analysis
   - Create histograms showing:
     - Image brightness distributions
     - Mask area distributions
     - Color channel statistics

In [ ]:
# ---------------------------------------------------------------------------- #
#                        PRINT CAR PIXEL RATIO IN IMAGES                       #
# ---------------------------------------------------------------------------- #

for idx in range(5):
    img_path = df_images_masks.iloc[idx]['image']
    mask_path = df_images_masks.iloc[idx]['mask']

    mask_array = np.array(Image.open(mask_path)) / 255

    unique_values, counts = np.unique(mask_array, return_counts = True)
    total_pixels = mask_array.size
    car_pixel_ratio = counts[1] / total_pixels

    print(f"Image {idx} foreground ratio: {car_pixel_ratio*100:.1f}%")

6. **Data Quality Checks**:
   - Identify any corrupted or missing files
   - Check for any images without corresponding masks
   - Verify mask quality (clean edges, no artifacts)
   - Look for potential data anomalies

In [ ]:
# # ---------------------------------------------------------------------------- #
# #                          CHECK FOR MISSING FILES                             #
# # ---------------------------------------------------------------------------- #

# missing_masks = []
# for image_path in train_set:
# 	# Derive mask path from image path
# 	mask_path = image_path.replace(TRAIN_IMAGES_DIR, TRAIN_MASKS_DIR).replace(".jpg", "_mask.gif")
# 	if not os.path.exists(mask_path):
# 		missing_masks.append(image_path)

# if missing_masks:
# 	print(f"Missing masks for {len(missing_masks)} images:")
# 	print(missing_masks[:5])  # Display first 5 missing masks
# else:
# 	print("All training images have corresponding masks.")

# # ---------------------------------------------------------------------------- #
# #                          VERIFY MASK QUALITY                                 #
# # ---------------------------------------------------------------------------- #

# corrupted_masks = []
# for mask_path in train_mask_set:
# 	try:
# 		mask = Image.open(mask_path)
# 		mask.verify()  # Verify if the file is not corrupted
# 	except Exception as e:
# 		corrupted_masks.append(mask_path)

# if corrupted_masks:
# 	print(f"Corrupted masks detected: {len(corrupted_masks)}")
# 	print(corrupted_masks[:5])  # Display first 5 corrupted masks
# else:
# 	print("No corrupted masks detected.")

# # ---------------------------------------------------------------------------- #
# #                          CHECK MASK ANOMALIES                                #
# # ---------------------------------------------------------------------------- #

# anomalous_masks = []
# for mask_path in train_mask_set:
# 	mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
# 	if mask is not None:
# 		unique_values = np.unique(mask)
# 		if len(unique_values) > 2 or not np.array_equal(unique_values, [0, 255]):
# 			anomalous_masks.append(mask_path)
# 	else:
# 		print(f"Mask could not be loaded: {mask_path}")

# if anomalous_masks:
# 	print(f"Anomalous masks detected: {len(anomalous_masks)}")
# 	print(anomalous_masks[:5])  # Display first 5 anomalous masks
# else:
# 	print("No anomalies detected in masks.")

## 4. Data Preprocessing and Augmentation

1. **Image Resizing Strategy**:
   - Determine target image size for training (e.g., 256x256, 512x512)
   - Consider memory constraints vs. model performance trade-offs
   - Implement resizing function that maintains aspect ratio
   - Handle both images and masks consistently
   - Choose appropriate interpolation methods (bilinear for images, nearest for masks)

In [ ]:
# ---------------------------------------------------------------------------- #
#                           IMAGE RESIZING FUNCTION                            #
# ---------------------------------------------------------------------------- #

def resize_images(image_paths, target_size, batch_size=500, save_dir="carvana-dataset/resized_images_512x512"):
    """
    Resize images to the specified target size in batches, then save to disk.

    Parameters:
        image_paths (list): List of file paths to the images to be resized.
        target_size (tuple): Target size as (width, height).
        batch_size (int): Number of images to process in each batch. Default is 500.
        save_dir (str): Directory to save resized images.

    Returns:
        str: Path to directory containing resized images.
    """
    # Check if directory already exists
    if os.path.exists(save_dir) and len(os.listdir(save_dir)) > 0:
        print(f"Directory '{save_dir}' already exists with files. Skipping resizing.")
        return save_dir
    
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    for i in range(0, len(image_paths), batch_size):
        batch = image_paths[i:i + batch_size]
        print(f"Processing image batch {i//batch_size + 1}/{(len(image_paths) + batch_size - 1)//batch_size}")
        
        for image_path in batch:
            try:
                with Image.open(image_path) as img:
                    resized_img = img.resize(target_size, Image.LANCZOS)    #type: ignore
                    # Save resized image
                    output_path = os.path.join(save_dir, os.path.basename(image_path))
                    resized_img.save(output_path)
            except Exception as e:
                print(f"Error processing {image_path}: {e}")
    
    print(f"Resized images saved to: {save_dir}")
    return save_dir

# ---------------------------------------------------------------------------- #
#                            MASK RESIZING FUNCTION                            #
# ---------------------------------------------------------------------------- #

def resize_masks(mask_paths, target_size, batch_size=500, save_dir="carvana-dataset/resized_masks_512x512"):
    """
    Resize masks to the specified target size in batches, then save to disk.

    Parameters:
        mask_paths (list): List of file paths to the masks to be resized.
        target_size (tuple): Target size as (width, height).
        batch_size (int): Number of masks to process in each batch. Default is 500.
        save_dir (str): Directory to save resized masks.

    Returns:
        str: Path to directory containing resized masks.
    """
    # Check if directory already exists
    if os.path.exists(save_dir) and len(os.listdir(save_dir)) > 0:
        print(f"Directory '{save_dir}' already exists with files. Skipping resizing.")
        return save_dir
    
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    for i in range(0, len(mask_paths), batch_size):
        batch = mask_paths[i:i + batch_size]
        print(f"Processing mask batch {i//batch_size + 1}/{(len(mask_paths) + batch_size - 1)//batch_size}")
        
        for mask_path in batch:
            try:
                with Image.open(mask_path) as mask:
                    resized_mask = mask.resize(target_size, Image.NEAREST)  #type: ignore
                    # Save resized mask
                    output_path = os.path.join(save_dir, os.path.basename(mask_path))
                    resized_mask.save(output_path)
            except Exception as e:
                print(f"Error processing {mask_path}: {e}")
    
    print(f"Resized masks saved to: {save_dir}")
    return save_dir

In [ ]:
# ---------------------------------------------------------------------------- #
#                        RESIZE THEN SAVE IMAGES & MASKS                       #
# ---------------------------------------------------------------------------- #

# Define target size
TARGET_SIZE = (512, 512)

# Process and save resized images (will skip if already exists)
resized_images_dir = resize_images(train_set, TARGET_SIZE, batch_size=200)
print(f"Images processed and saved to: {resized_images_dir}")

# Process and save resized masks (will skip if already exists)
resized_masks_dir = resize_masks(train_mask_set, TARGET_SIZE, batch_size=200)
print(f"Masks processed and saved to: {resized_masks_dir}")

In [ ]:
# Display the first 5 resized images and masks
resized_images = [Image.open(os.path.join(resized_images_dir, os.path.basename(image_path))) for image_path in train_set[:5]]
resized_masks = [Image.open(os.path.join(resized_masks_dir, os.path.basename(mask_path))) for mask_path in train_mask_set[:5]]

fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# Display images
for ax, img in zip(axes[0], resized_images):
    ax.imshow(img)
    ax.set_title("Image")
    ax.axis('off')

# Display masks
for ax, mask in zip(axes[1], resized_masks):
    ax.imshow(mask, cmap='gray')
    ax.set_title("Mask")
    ax.axis('off')

plt.tight_layout()
plt.show()

2. **Normalization Techniques**:
   - Normalize pixel values to [0, 1] range by dividing by 255
   - Consider alternative normalization strategies:
     - Standard normalization (mean=0, std=1)
     - Per-channel normalization using ImageNet statistics
   - Ensure masks remain binary (0 and 1) after preprocessing

In [ ]:
# ---------------------------------------------------------------------------- #
#                        NORMALIZATION FUNCTIONS                               #
# ---------------------------------------------------------------------------- #

def normalize_images(image_paths, target_size=(512, 512), batch_size=500, save_dir="carvana-dataset/normalized_images_512x512"):
    """
    Load, resize, and normalize images to [0, 1] range in batches, then save to disk.

    Parameters:
        image_paths (list): List of file paths to the images to be normalized.
        target_size (tuple): Target size as (width, height).
        batch_size (int): Number of images to process in each batch. Default is 500.
        save_dir (str): Directory to save normalized images.

    Returns:
        str: Path to directory containing normalized images.
    """
    # Check if directory already exists
    if os.path.exists(save_dir) and len(os.listdir(save_dir)) > 0:
        print(f"Directory '{save_dir}' already exists with files. Skipping normalization.")
        return save_dir
    
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    for i in range(0, len(image_paths), batch_size):
        batch = image_paths[i:i + batch_size]
        print(f"Processing image batch {i//batch_size + 1}/{(len(image_paths) + batch_size - 1)//batch_size}")
        
        for image_path in batch:
            try:
                with Image.open(image_path) as img:
                    # Resize and convert to RGB
                    resized_img = img.resize(target_size, Image.LANCZOS).convert('RGB') #type: ignore
                    # Convert to numpy array and normalize
                    img_array = np.array(resized_img).astype(np.float32) / 255.0
                    
                    # Save normalized image as numpy array
                    output_filename = os.path.splitext(os.path.basename(image_path))[0] + '.npy'
                    output_path = os.path.join(save_dir, output_filename)
                    np.save(output_path, img_array)
                    
            except Exception as e:
                print(f"Error processing {image_path}: {e}")
    
    print(f"Normalized images saved to: {save_dir}")
    return save_dir

def normalize_masks(mask_paths, target_size=(512, 512), batch_size=500, save_dir="carvana-dataset/normalized_masks_512x512"):
    """
    Load, resize, and normalize masks to binary [0, 1] values in batches, then save to disk.

    Parameters:
        mask_paths (list): List of file paths to the masks to be normalized.
        target_size (tuple): Target size as (width, height).
        batch_size (int): Number of masks to process in each batch. Default is 500.
        save_dir (str): Directory to save normalized masks.

    Returns:
        str: Path to directory containing normalized masks.
    """
    # Check if directory already exists
    if os.path.exists(save_dir) and len(os.listdir(save_dir)) > 0:
        print(f"Directory '{save_dir}' already exists with files. Skipping normalization.")
        return save_dir
    
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    for i in range(0, len(mask_paths), batch_size):
        batch = mask_paths[i:i + batch_size]
        print(f"Processing mask batch {i//batch_size + 1}/{(len(mask_paths) + batch_size - 1)//batch_size}")
        
        for mask_path in batch:
            try:
                with Image.open(mask_path) as mask:
                    # Resize and convert to grayscale
                    resized_mask = mask.resize(target_size, Image.NEAREST).convert('L') #type: ignore
                    # Convert to numpy array and make binary
                    mask_array = np.array(resized_mask)
                    binary_mask = (mask_array > 127).astype(np.float32)
                    
                    # Save normalized mask as numpy array
                    output_filename = os.path.splitext(os.path.basename(mask_path))[0] + '.npy'
                    output_path = os.path.join(save_dir, output_filename)
                    np.save(output_path, binary_mask)
                    
            except Exception as e:
                print(f"Error processing {mask_path}: {e}")
    
    print(f"Normalized masks saved to: {save_dir}")
    return save_dir

In [ ]:
# ---------------------------------------------------------------------------- #
#                     NORMALIZE AND SAVE IMAGES & MASKS                        #
# ---------------------------------------------------------------------------- #

# Define target size
TARGET_SIZE = (512, 512)

# Process and save normalized images (will skip if already exists)
images_dir = normalize_images(train_set, TARGET_SIZE, batch_size=200)
print(f"Images processed and saved to: {images_dir}")

# Process and save normalized masks (will skip if already exists)
masks_dir = normalize_masks(train_mask_set, TARGET_SIZE, batch_size=200)
print(f"Masks processed and saved to: {masks_dir}")

In [ ]:
# Display the first 5 normalized images and masks
normalized_images = [np.load(os.path.join(images_dir, os.path.splitext(os.path.basename(image_path))[0] + '.npy')) for image_path in train_set[:5]]
normalized_masks = [np.load(os.path.join(masks_dir, os.path.splitext(os.path.basename(mask_path))[0] + '.npy')) for mask_path in train_mask_set[:5]]

fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# Display normalized images
for ax, img in zip(axes[0], normalized_images):
    ax.imshow(img)
    ax.set_title("Normalized Image")
    ax.axis('off')

# Display normalized masks
for ax, mask in zip(axes[1], normalized_masks):
    ax.imshow(mask, cmap='gray')
    ax.set_title("Normalized Mask")
    ax.axis('off')

plt.tight_layout()
plt.show()

3. **Data Loading Pipeline**:
   - Create efficient data generator/loader functions
   - Implement batch loading for memory efficiency
   - Set up train/validation split (e.g., 80/20 or 90/10)
   - Ensure reproducible splits using fixed random seeds
   - Handle file path management and error checking

In [ ]:
# ---------------------------------------------------------------------------- #
#                              RESIZED DATA LOADER                             #
# ---------------------------------------------------------------------------- #

def load_resized_data(images_dir, masks_dir, limit=None):
    """
    Load resized images and masks from the specified directories.

    Parameters:
        images_dir (str): Directory containing resized images.
        masks_dir (str): Directory containing resized masks.
        limit (int, optional): Maximum number of images and masks to load. Default is None (load all).

    Returns:
        tuple: Two lists containing loaded images and masks as numpy arrays.
    """
    # Get sorted lists of image and mask file paths
    image_paths = sorted(glob.glob(os.path.join(images_dir, "*.*")))
    mask_paths = sorted(glob.glob(os.path.join(masks_dir, "*.*")))

    # Apply limit if specified
    if limit:
        image_paths = image_paths[:limit]
        mask_paths = mask_paths[:limit]

    # Load images and masks
    images = [np.array(Image.open(img_path)) for img_path in image_paths]
    masks = [np.array(Image.open(mask_path)) for mask_path in mask_paths]

    return images, masks

# ---------------------------------------------------------------------------- #
#                            NORMALIZED DATA LOADER                            #
# ---------------------------------------------------------------------------- #
def load_normalized_data(images_dir, masks_dir, limit=None):
    """
    Load normalized images and masks from saved numpy files.
    
    Parameters:
        images_dir (str): Directory containing normalized image .npy files
        masks_dir (str): Directory containing normalized mask .npy files
        limit (int, optional): Maximum number of images and masks to load. Default is None (load all).
    
    Returns:
        tuple: (normalized_images, normalized_masks) as numpy arrays
    """
    print("Loading normalized images...")
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.npy')])
    if limit:
        image_files = image_files[:limit]
    normalized_images = []
    for img_file in image_files:
        img_array = np.load(os.path.join(images_dir, img_file))
        normalized_images.append(img_array)
    
    print("Loading normalized masks...")
    mask_files = sorted([f for f in os.listdir(masks_dir) if f.endswith('.npy')])
    if limit:
        mask_files = mask_files[:limit]
    normalized_masks = []
    for mask_file in mask_files:
        mask_array = np.load(os.path.join(masks_dir, mask_file))
        normalized_masks.append(mask_array)
    
    return np.array(normalized_images), np.array(normalized_masks)

In [ ]:
resized_images, resized_masks = load_resized_data(resized_images_dir, resized_masks_dir, limit=100)
print(f"Loaded {len(resized_images)} images and {len(resized_masks)} masks.")

normalized_images, normalized_masks = load_normalized_data(images_dir, masks_dir, limit=100)
print(f"Loaded {len(normalized_images)} images and {len(normalized_masks)} masks.")

In [ ]:
# Count of resized data
resized_images_count = len(os.listdir(resized_images_dir))
resized_masks_count = len(os.listdir(resized_masks_dir))
print(f"Resized images count: {resized_images_count}")
print(f"Resized masks count: {resized_masks_count}")

# Count of normalized data
normalized_images_count = len(os.listdir(images_dir))
normalized_masks_count = len(os.listdir(masks_dir))
print(f"Normalized images count: {normalized_images_count}")
print(f"Normalized masks count: {normalized_masks_count}")

# Compare counts with original train images and masks
if resized_images_count == len(train_set) and resized_masks_count == len(train_mask_set) and \
    normalized_images_count == len(train_set) and normalized_masks_count == len(train_mask_set):
     print("Verification successful: Resized and normalized counts match the original train images and masks count.")
else:
     print("Verification failed: Counts do not match.")

4. **Data Augmentation Strategies**:
   - **Geometric Transformations**:
     - Horizontal and vertical flipping
     - Random rotation (small angles, e.g., ±30 degrees)
     - Random cropping and scaling
     - Elastic deformation for more robust training (Useful in medical domain)
   
   - **Photometric Augmentations**:
     - Brightness adjustment (±20%)
     - Contrast modification
     - Color jittering (hue, saturation adjustments)
     - Gaussian noise addition
   
   - **Advanced Augmentations**:
     - Random shadow and highlight simulation
     - Perspective transformation
     - Motion blur effects
     - Weather condition simulation

   - **Chosen Augmentation Settings**

     - **Geometric Transformations**
         - Horizontal and Vertical Flipping <br>
         - ```30``` Degrees Rotation <br>
         - Zoom ```0.8 - 1.2``` <br>

     - **Photometric Augmentations**
         - Brightness ```0.7 - 1.3``` <br>
         - Contrast ```0.7 - 1.3``` <br>
         - Gaussian Noise (Mean 0) ```0.01 - 0.05``` <br>
         - Gamma Range ```0.8 - 1.2``` <br>

     - **Advanced Augmentations**
         - Gaussian Blur ```0.1 - 0.7``` <br>
         - Sharpening Alpha ```0.1 - 0.3``` <br>

5. **Augmentation Implementation Guidelines**:
   - Apply same geometric transformations to both image and mask
   - Only apply photometric changes to images, not masks
   - Use appropriate probability values for each augmentation
   - Create augmentation pipeline using libraries like Albumentations
   - Ensure augmented masks remain binary

In [ ]:
# ---------------------------------------------------------------------------- #
#                      ALBUMENTATIONS AUGMENTATION PIPELINE                    #
# ---------------------------------------------------------------------------- #

def create_training_augmentation():
    """
    Create training augmentation pipeline using Albumentations
    Applies geometric, photometric, and advanced augmentations
    """
    transforms = [
        # Geometric Transformations (affect both image and mask)
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.Rotate(limit=30, p=0.7, border_mode=cv2.BORDER_REFLECT),
        A.RandomScale(scale_limit=0.2, p=0.6),  # This gives us 0.8-1.2 zoom range
        
        # Photometric Augmentations (affect only image)
        A.RandomBrightnessContrast(
            brightness_limit=0.3,  # 0.7-1.3 range
            contrast_limit=0.3,    # 0.7-1.3 range
            p=0.6
        ),
        A.GaussNoise(
            var_limit=(0.01**2, 0.05**2),  # Convert std to variance    #type: ignore
            mean=0, #type: ignore
            p=0.5
        ),
        A.RandomGamma(
            gamma_limit=(80, 120),  # 0.8-1.2 range (multiplied by 100)
            p=0.6
        ),
        
        # Advanced Augmentations (affect only image)
        A.GaussianBlur(
            blur_limit=(3, 7),  # Kernel size range
            sigma_limit=(0.1, 0.7),
            p=0.4
        ),
        A.Sharpen(
            alpha=(0.1, 0.3),
            lightness=(0.8, 1.2),
            p=0.4
        ),
        
        # Normalization (always applied)
        A.Normalize(
            mean=[0.0, 0.0, 0.0],   #type: ignore
            std=[1.0, 1.0, 1.0],    #type: ignore
            max_pixel_value=255.0,
            p=1.0
        )
    ]
    
    return A.Compose(transforms)

def create_validation_augmentation():
    """
    Create validation augmentation pipeline (only normalization)
    """
    transforms = [
        A.Normalize(
            mean=[0.0, 0.0, 0.0],   #type: ignore
            std=[1.0, 1.0, 1.0],    #type: ignore
            max_pixel_value=255.0,
            p=1.0
        )
    ]
    
    return A.Compose(transforms)    #type: ignore

In [ ]:
# ---------------------------------------------------------------------------- #
#                     ALBUMENTATIONS DATA GENERATOR CLASS                      #
# ---------------------------------------------------------------------------- #

class AlbumentationsDataGenerator(tf.keras.utils.Sequence):
    """
    Custom data generator using Albumentations for augmentation
    Inherits from tf.keras.utils.Sequence for compatibility with model.fit()
    """
    def __init__(self, images_dir, masks_dir, batch_size=8, target_size=(512, 512), 
                 augmentation=None, shuffle=True):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.augmentation = augmentation
        self.shuffle = shuffle
        
        # Get file lists
        self.image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.npy')])
        
        # For masks, we need to derive the corresponding mask filename from the image filename
        # Mask files have '_mask' suffix: image_file.npy -> image_file_mask.npy
        self.mask_files = [f.replace('.npy', '_mask.npy') for f in self.image_files]
        
        # Verify that all corresponding mask files exist
        missing_masks = []
        for mask_file in self.mask_files:
            if not os.path.exists(os.path.join(masks_dir, mask_file)):
                missing_masks.append(mask_file)
        
        if missing_masks:
            raise FileNotFoundError(f"Missing mask files: {missing_masks[:5]}...")
        
        assert len(self.image_files) == len(self.mask_files), "Number of images and masks must match"
        
        self.indices = list(range(len(self.image_files)))
        self.on_epoch_end()
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indices) // self.batch_size
    
    def __getitem__(self, index):
        """Generate one batch of data"""
        # Generate indices of the batch
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        
        # Generate data
        X, y = self.__data_generation(batch_indices)
        
        return X, y
    
    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __data_generation(self, batch_indices):
        """Generates data containing batch_size samples"""
        # Initialize batch arrays
        X = np.empty((self.batch_size, *self.target_size, 3), dtype=np.float32)
        y = np.empty((self.batch_size, *self.target_size, 1), dtype=np.float32)
        
        # Generate data
        for i, idx in enumerate(batch_indices):
            # Load image and mask
            image_path = os.path.join(self.images_dir, self.image_files[idx])
            mask_path = os.path.join(self.masks_dir, self.mask_files[idx])
            
            image = np.load(image_path)
            mask = np.load(mask_path)
            
            # Convert to uint8 for Albumentations
            if image.dtype != np.uint8:
                image = (image * 255).astype(np.uint8)
            if mask.dtype != np.uint8:
                mask = (mask * 255).astype(np.uint8)
            
            # Resize if necessary
            if image.shape[:2] != self.target_size:
                image = cv2.resize(image, self.target_size, interpolation=cv2.INTER_LINEAR)
                mask = cv2.resize(mask, self.target_size, interpolation=cv2.INTER_NEAREST)
            
            # Apply augmentation
            if self.augmentation:
                augmented = self.augmentation(image=image, mask=mask)
                image = augmented['image']
                mask = augmented['mask']
                
                # Ensure augmented image and mask are back to target size
                if image.shape[:2] != self.target_size:
                    image = cv2.resize(image, self.target_size, interpolation=cv2.INTER_LINEAR)
                    mask = cv2.resize(mask, self.target_size, interpolation=cv2.INTER_NEAREST)
            else:
                # Normalize if no augmentation pipeline
                image = image.astype(np.float32) / 255.0
                mask = mask.astype(np.float32) / 255.0
            
            # Ensure mask is binary
            mask = (mask > 0.5).astype(np.float32)
            
            # Add channel dimension to mask if needed
            if len(mask.shape) == 2:
                mask = np.expand_dims(mask, axis=-1)
            
            X[i] = image
            y[i] = mask
        
        return X, y

In [ ]:
# ---------------------------------------------------------------------------- #
#                     TENSORFLOW DATASET WITH ALBUMENTATIONS                   #
# ---------------------------------------------------------------------------- #

def create_albumentations_dataset(images_dir, masks_dir, batch_size=8, target_size=(512, 512), 
                                  augmentation=None, shuffle=True):
    """
    Create TensorFlow dataset with Albumentations augmentation
    """
    def load_and_augment(image_path, mask_path):
        # Load files
        image = np.load(image_path.numpy().decode('utf-8'))
        mask = np.load(mask_path.numpy().decode('utf-8'))
        
        # Convert to uint8
        if image.dtype != np.uint8:
            image = (image * 255).astype(np.uint8)
        if mask.dtype != np.uint8:
            mask = (mask * 255).astype(np.uint8)
        
        # Resize if necessary
        if image.shape[:2] != target_size:
            image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
            mask = cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST)
        
        # Apply augmentation
        if augmentation:
            augmented = augmentation(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        else:
            image = image.astype(np.float32) / 255.0
            mask = mask.astype(np.float32) / 255.0
        
        # Ensure mask is binary and has correct shape
        mask = (mask > 0.5).astype(np.float32)
        if len(mask.shape) == 2:
            mask = np.expand_dims(mask, axis=-1)
        
        return image.astype(np.float32), mask.astype(np.float32)
    
    # Get file paths
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.npy')])
    mask_files = sorted([f for f in os.listdir(masks_dir) if f.endswith('.npy')])
    
    image_paths = [os.path.join(images_dir, f) for f in image_files]
    mask_paths = [os.path.join(masks_dir, f) for f in mask_files]
    
    # Create dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    
    dataset = dataset.map(
        lambda x, y: tf.py_function(
            load_and_augment, 
            [x, y], 
            [tf.float32, tf.float32]
        ),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    # Set shapes explicitly
    dataset = dataset.map(lambda x, y: (
        tf.reshape(x, (*target_size, 3)),
        tf.reshape(y, (*target_size, 1))
    ))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset

In [ ]:
# ---------------------------------------------------------------------------- #
#                       DEMONSTRATION WITH ALBUMENTATIONS                      #
# ---------------------------------------------------------------------------- #

def demonstrate_albumentations_augmentations(images_dir, masks_dir, num_samples=3):
    """Demonstrate Albumentations augmentation effects"""
    
    # Create augmentation pipeline
    aug_pipeline = create_training_augmentation()
    
    # Load sample images
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.npy')])[:num_samples]
    mask_files = sorted([f for f in os.listdir(masks_dir) if f.endswith('.npy')])[:num_samples]
    
    fig, axes = plt.subplots(num_samples, 4, figsize=(16, 4 * num_samples))
    
    for i, (img_file, mask_file) in enumerate(zip(image_files, mask_files)):
        # Load original
        original_image = np.load(os.path.join(images_dir, img_file))
        original_mask = np.load(os.path.join(masks_dir, mask_file))
        
        # Convert to uint8 for Albumentations
        image_uint8 = (original_image * 255).astype(np.uint8)
        mask_uint8 = (original_mask * 255).astype(np.uint8)
        
        # Apply augmentation
        augmented = aug_pipeline(image=image_uint8, mask=mask_uint8)
        aug_image = augmented['image']
        aug_mask = augmented['mask']
        
        # Display results
        axes[i, 0].imshow(original_image)
        axes[i, 0].set_title('Original Image')
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(original_mask, cmap='gray')
        axes[i, 1].set_title('Original Mask')
        axes[i, 1].axis('off')
        
        axes[i, 2].imshow(aug_image)
        axes[i, 2].set_title('Augmented Image')
        axes[i, 2].axis('off')
        
        axes[i, 3].imshow(aug_mask, cmap='gray')
        axes[i, 3].set_title('Augmented Mask')
        axes[i, 3].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# ---------------------------------------------------------------------------- #
#                                DATA GENERATORS                               #
# ---------------------------------------------------------------------------- #

def create_data_generators(images_dir, masks_dir, batch_size=8, target_size=(512, 512), 
                          validation_split=0.2):
    """
    Create training and validation data generators using Albumentations
    """
    # Get all file indices
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.npy')])
    total_samples = len(image_files)
    
    # Split indices
    val_samples = int(total_samples * validation_split)
    train_samples = total_samples - val_samples
    
    # Create file lists for train and validation
    train_files = image_files[:train_samples]
    val_files = image_files[train_samples:]
    
    # Create augmentation pipelines
    train_aug = create_training_augmentation()
    val_aug = create_validation_augmentation()
    
    # Create generators
    train_generator = AlbumentationsDataGenerator(
        images_dir=images_dir,
        masks_dir=masks_dir,
        batch_size=batch_size,
        target_size=target_size,
        augmentation=train_aug,
        shuffle=True
    )
    
    val_generator = AlbumentationsDataGenerator(
        images_dir=images_dir,
        masks_dir=masks_dir,
        batch_size=batch_size,
        target_size=target_size,
        augmentation=val_aug,
        shuffle=False
    )
    
    # Adjust file lists
    train_generator.image_files = train_files
    train_generator.mask_files = [f.replace('.npy', '_mask.npy') for f in train_files]
    train_generator.indices = list(range(len(train_files)))
    
    val_generator.image_files = val_files
    val_generator.mask_files = [f.replace('.npy', '_mask.npy') for f in val_files]
    val_generator.indices = list(range(len(val_files)))
    
    return train_generator, val_generator

6. **Validation Strategy**:
   - Keep validation set without augmentation for fair evaluation
   - Implement stratified splitting to ensure balanced validation
   - Consider cross-validation for more robust model evaluation
   - Monitor augmentation effects on training convergence

In [ ]:
# Create training and validation generators
train_gen, val_gen = create_data_generators(
    images_dir=images_dir, 
    masks_dir=masks_dir,
    batch_size=16,
    target_size=(512, 512),
    validation_split=0.2
)

# Demonstrate augmentations
demonstrate_albumentations_augmentations(images_dir, masks_dir, num_samples=3)

## 5. U-Net Architecture Implementation

### Understanding U-Net Architecture:

U-Net consists of two main paths:
- **Contracting Path (Encoder)**: Captures context through downsampling
- **Expanding Path (Decoder)**: Enables precise localization through upsampling
- **Skip Connections**: Combine low-level and high-level features

### Step-by-Step Implementation:

1. **Encoder (Contracting Path) Design**:
   - **Input Layer**: Accept images of size (height, width, 3)
   - **Encoder Block Structure**:
     - Two 3x3 convolutions with ReLU activation
     - Batch normalization after each convolution
     - Dropout for regularization (0.1-0.2)
     - 2x2 max pooling for downsampling
   
   - **Encoder Levels**:
     - Level 1: 64 filters, input size
     - Level 2: 128 filters, 1/2 input size
     - Level 3: 256 filters, 1/4 input size
     - Level 4: 512 filters, 1/8 input size

2. **Bottleneck (Bridge) Design**:
   - Located at the lowest resolution (1/16 input size)
   - Two 3x3 convolutions with 1024 filters
   - Higher dropout rate (0.3-0.5) for regularization
   - No max pooling (preparation for upsampling)

3. **Decoder (Expanding Path) Design**:
   - **Decoder Block Structure**:
     - 2x2 transpose convolution for upsampling
     - Concatenation with corresponding encoder features (skip connections)
     - Two 3x3 convolutions with ReLU activation
     - Batch normalization and dropout
   
   - **Decoder Levels**:
     - Level 1: Upsample to 1/8, concatenate with encoder level 4
     - Level 2: Upsample to 1/4, concatenate with encoder level 3
     - Level 3: Upsample to 1/2, concatenate with encoder level 2
     - Level 4: Upsample to full size, concatenate with encoder level 1

4. **Skip Connections Implementation**:
   - Store encoder feature maps at each level
   - Concatenate along channel dimension in decoder
   - Ensure spatial dimensions match for concatenation
   - Handle any size mismatches through cropping or padding

5. **Output Layer Design**:
   - Final 1x1 convolution to reduce channels to number of classes
   - For binary segmentation: 1 output channel
   - Sigmoid activation for binary classification
   - Ensure output size matches input size

6. **Advanced Architecture Considerations**:
   - **Residual Connections**: Add within encoder/decoder blocks
   - **Attention Mechanisms**: Focus on important features
   - **Deep Supervision**: Add auxiliary outputs at multiple scales
   - **Dense Connections**: Connect all previous layers

### Architecture Benefits:
- **Skip Connections**: Preserve spatial information lost during downsampling
- **Symmetric Design**: Balanced encoder-decoder structure
- **Multi-scale Features**: Combines different levels of abstraction
- **End-to-End Training**: Single network for complete segmentation pipeline

### Implementation Tips:
- Use appropriate padding to maintain spatial dimensions
- Consider memory requirements when choosing filter numbers
- Experiment with different activation functions and normalizations
- Monitor gradient flow through the deep network

In [ ]:
# ---------------------------------------------------------------------------- #
#                ENCODER BLOCK LEVEL 1 (64 FILTERS, INPUT SIZE)                #
# ---------------------------------------------------------------------------- #

inputs = Input(shape=(512,512, 3))

encoding_block_1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
encoding_block_1 = BatchNormalization()(encoding_block_1)

encoding_block_1 = Conv2D(64, 3, activation='relu', padding='same')(encoding_block_1)
encoding_block_1 = BatchNormalization()(encoding_block_1)

encoding_block_1 = Dropout(0.1)(encoding_block_1)
encoding_block_1_pooling = MaxPooling2D((2, 2))(encoding_block_1)  # Downsampling

# ---------------------------------------------------------------------------- #
#              ENCODER BLOCK LEVEL 2 (128 FILTERS, 1/2 INPUT SIZE)             #
# ---------------------------------------------------------------------------- #

encoding_block_2 = Conv2D(128, 3, activation='relu', padding='same')(encoding_block_1_pooling)
encoding_block_2 = BatchNormalization()(encoding_block_2)

encoding_block_2 = Conv2D(128, 3, activation='relu', padding='same')(encoding_block_2)
encoding_block_2 = BatchNormalization()(encoding_block_2)

encoding_block_2 = Dropout(0.1)(encoding_block_2)
encoding_block_2_pooling = MaxPooling2D((2, 2))(encoding_block_2)  # Downsampling

# ---------------------------------------------------------------------------- #
#              ENCODER BLOCK LEVEL 3 (256 FILTERS, 1/4 INPUT SIZE)             #
# ---------------------------------------------------------------------------- #

encoding_block_3 = Conv2D(256, 3, activation='relu', padding='same')(encoding_block_2_pooling)
encoding_block_3 = BatchNormalization()(encoding_block_3)

encoding_block_3 = Conv2D(256, 3, activation='relu', padding='same')(encoding_block_3)
encoding_block_3 = BatchNormalization()(encoding_block_3)

encoding_block_3 = Dropout(0.1)(encoding_block_3)
encoding_block_3_pooling = MaxPooling2D((2, 2))(encoding_block_3)  # Downsampling

# ---------------------------------------------------------------------------- #
#              ENCODER BLOCK LEVEL 4 (512 FILTERS, 1/8 INPUT SIZE)             #
# ---------------------------------------------------------------------------- #

encoding_block_4 = Conv2D(512, 3, activation='relu', padding='same')(encoding_block_3_pooling)
encoding_block_4 = BatchNormalization()(encoding_block_4)

encoding_block_4 = Conv2D(512, 3, activation='relu', padding='same')(encoding_block_4)
encoding_block_4 = BatchNormalization()(encoding_block_4)

encoding_block_4 = Dropout(0.1)(encoding_block_4)
encoding_block_4_pooling = MaxPooling2D((2, 2))(encoding_block_4)  # Downsampling


# ---------------------------------------------------------------------------- #
#                BOTTLENECK BRIDGE 1024 FILTERS, 1/16 INPUT SIZE               #
# ---------------------------------------------------------------------------- #

bottleneck_bridge = Conv2D(1024, 3, activation='relu', padding='same')(encoding_block_4_pooling)
bottleneck_bridge = BatchNormalization()(bottleneck_bridge)

bottleneck_bridge = Conv2D(1024, 3, activation='relu', padding='same')(bottleneck_bridge)
bottleneck_bridge = BatchNormalization()(bottleneck_bridge)

bottleneck_bridge = Dropout(0.3)(bottleneck_bridge) # No downsampling after

# ---------------------------------------------------------------------------- #
#              DECODER BLOCK LEVEL 1 (512 FILTERS, 1/8 INPUT SIZE)             #
# ---------------------------------------------------------------------------- #

decoder_block_1 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(bottleneck_bridge) 
decoder_block_1 = concatenate([decoder_block_1, encoding_block_4]) # Concatenate with Encoder Block Level 4

decoder_block_1 = Conv2D(512, 3, activation='relu', padding='same')(decoder_block_1)
decoder_block_1 = BatchNormalization()(decoder_block_1)

decoder_block_1 = Conv2D(512, 3, activation='relu', padding='same')(decoder_block_1)
decoder_block_1 = BatchNormalization()(decoder_block_1)

decoder_block_1 = Dropout(0.2)(decoder_block_1)

# ---------------------------------------------------------------------------- #
#              DECODER BLOCK LEVEL 2 (256 FILTERS, 1/4 INPUT SIZE)             #
# ---------------------------------------------------------------------------- #

decoder_block_2 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(decoder_block_1)
decoder_block_2 = concatenate([decoder_block_2, encoding_block_3]) # Concatenate with Encoder Block Level 3

decoder_block_2 = Conv2D(256, 3, activation='relu', padding='same')(decoder_block_2)
decoder_block_2 = BatchNormalization()(decoder_block_2)

decoder_block_2 = Conv2D(256, 3, activation='relu', padding='same')(decoder_block_2)
decoder_block_2 = BatchNormalization()(decoder_block_2)

decoder_block_2 = Dropout(0.2)(decoder_block_2)

# ---------------------------------------------------------------------------- #
#              DECODER BLOCK LEVEL 3 (128 FILTERS, 1/2 INPUT SIZE)             #
# ---------------------------------------------------------------------------- #

decoder_block_3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(decoder_block_2)
decoder_block_3 = concatenate([decoder_block_3, encoding_block_2]) # Concatenate with Encoder Block Level 2

decoder_block_3 = Conv2D(128, 3, activation='relu', padding='same')(decoder_block_3)
decoder_block_3 = BatchNormalization()(decoder_block_3)

decoder_block_3 = Conv2D(128, 3, activation='relu', padding='same')(decoder_block_3)
decoder_block_3 = BatchNormalization()(decoder_block_3)

decoder_block_3 = Dropout(0.1)(decoder_block_3)

# ---------------------------------------------------------------------------- #
#                DECODER BLOCK LEVEL 4 (64 FILTERS, INPUT SIZE)                #
# ---------------------------------------------------------------------------- #

decoder_block_4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(decoder_block_3)
decoder_block_4 = concatenate([decoder_block_4, encoding_block_1]) # Concatenate with Encoder Block Level 1

decoder_block_4 = Conv2D(64, 3, activation='relu', padding='same')(decoder_block_4)
decoder_block_4 = BatchNormalization()(decoder_block_4)

decoder_block_4 = Conv2D(64, 3, activation='relu', padding='same')(decoder_block_4)
decoder_block_4 = BatchNormalization()(decoder_block_4)

decoder_block_4 = Dropout(0.1)(decoder_block_4)

# ---------------------------------------------------------------------------- #
#                                 OUTPUT LAYER                                 #
# ---------------------------------------------------------------------------- #

outputs = Conv2D(1, 1, activation = 'sigmoid')(decoder_block_4)

# Create the model
model = Model(inputs = inputs, outputs = outputs)

## 6. Model Compilation and Training Setup

1. **Loss Function Selection**:   
   - **Combined Loss**: Binary cross-entropy + Dice loss
     - Balances pixel-wise accuracy and overlap
     - Often provides best results for segmentation

2. **Optimizer Configuration**:
   - **Adam Optimizer**: Good default choice
     - Learning rate: 1e-3 to 1e-4
     - Beta1: 0.9, Beta2: 0.999
     - Adaptive learning rates for each parameter
      
   - **Learning Rate Scheduling**:
     - ReduceLROnPlateau: Reduce when validation loss plateaus
     - CosineAnnealingLR: Cyclical learning rate changes
     - ExponentialDecay: Gradual learning rate reduction

3. **Evaluation Metrics Setup**:
   - **IoU (Intersection over Union)**:
     - Primary metric for segmentation tasks
     - Measures overlap between prediction and ground truth
     - Range: 0 to 1 (higher is better)
   
   - **Dice Coefficient**:
     - Similar to IoU but with different mathematical formulation
     - Also measures overlap quality
     - More sensitive to small objects
   
   - **Pixel Accuracy**:
     - Percentage of correctly classified pixels
     - Can be misleading with class imbalance
   
   - **Precision and Recall**:
     - Precision: True positives / (True positives + False positives)
     - Recall: True positives / (True positives + False negatives)

4. **Callback Configuration**:
   - **ModelCheckpoint**:
     - Save best model based on validation metric
     - Monitor validation IoU or validation loss
     - Save only when improvement is detected
     - Keep backup of best weights
   
   - **EarlyStopping**:
     - Stop training when validation metric stops improving
     - Patience: 10-20 epochs
     - Restore best weights when stopping
   
   - **ReduceLROnPlateau**:
     - Reduce learning rate when training plateaus
     - Factor: 0.5 (reduce by half)
     - Patience: 5-10 epochs
     - Minimum learning rate: 1e-7
   
   - **CSVLogger**:
     - Log training metrics to CSV file
     - Useful for post-training analysis
     - Track loss and metric evolution

5. **Training Configuration**:
   - **Batch Size Selection**:
     - Balance between memory constraints and training stability
     - Typical values: 8, 16, 32 (depending on GPU memory)
     - Smaller batches may need adjusted learning rate
   
   - **Epoch Planning**:
     - Start with 50-100 epochs
     - Monitor for overfitting or underfitting
     - Adjust based on learning curves
   
   - **Validation Strategy**:
     - Use separate validation set (not used in training)
     - Evaluate every epoch
     - Monitor both loss and custom metrics

### Best Practices:
- Start with proven hyperparameters and adjust gradually
- Monitor training curves to detect overfitting early
- Use mixed precision training for memory efficiency
- Implement gradient clipping if training becomes unstable
- Save model architecture separately from weights

In [ ]:
# ---------------------------------------------------------------------------- #
#                            COMBINED LOSS FUNCTION                            #
# ---------------------------------------------------------------------------- #

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """
    Calculate Dice coefficient for binary segmentation
    
    Args:
        y_true: Ground truth masks
        y_pred: Predicted masks
        smooth: Smoothing factor to avoid division by zero
    
    Returns:
        Dice coefficient value
    """
    y_true_f = tf.keras.backend.flatten(y_true) #type: ignore
    y_pred_f = tf.keras.backend.flatten(y_pred) #type: ignore
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)    #type: ignore
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)    #type: ignore

def dice_loss(y_true, y_pred):
    """
    Dice loss function (1 - Dice coefficient)
    """
    return 1 - dice_coefficient(y_true, y_pred)

def combined_loss(alpha=0.5):
    """
    Combined loss function: Binary Cross-Entropy + Dice Loss
    
    Args:
        alpha: Weight factor for combining losses (0.5 = equal weight)
               alpha closer to 0 = more weight to BCE
               alpha closer to 1 = more weight to Dice
    
    Returns:
        Combined loss function
    """
    def loss(y_true, y_pred):
        bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)  #type: ignore
        dice_loss_value = dice_loss(y_true, y_pred)
        return alpha * dice_loss_value + (1 - alpha) * bce_loss
    
    return loss

In [ ]:
# ---------------------------------------------------------------------------- #
#                          LEARNING RATE SCHEDULING                            #
# ---------------------------------------------------------------------------- #

def create_learning_rate_scheduler(initial_lr=1e-3, schedule_type='plateau'):
    """
    Create different types of learning rate schedulers
    
    Args:
        initial_lr: Starting learning rate
        schedule_type: Type of scheduler ('plateau', 'exponential', 'cosine', 'step')
    
    Returns:
        Learning rate scheduler callback
    """
    
    if schedule_type == 'plateau':
        # Reduces LR when validation loss stops improving
        return callbacks.ReduceLROnPlateau(
            monitor='val_loss',           # Metric to monitor
            factor=0.5,                   # Factor to reduce LR (new_lr = lr * factor)
            patience=5,                   # Number of epochs to wait before reducing
            min_lr=1e-7,                 # Minimum learning rate    #type: ignore
            verbose=1,                    # Print message when LR is reduced
            cooldown=2                    # Epochs to wait after LR reduction
        )
    
    elif schedule_type == 'exponential':
        # Exponentially decay learning rate
        return callbacks.LearningRateScheduler(
            lambda epoch: initial_lr * (0.95 ** epoch),  # Decay by 5% each epoch
            verbose=1
        )
    
    elif schedule_type == 'cosine':
        # Cosine annealing - gradually decreases then increases
        def cosine_schedule(epoch, lr):
            import math
            epochs = 100  # Total expected epochs
            return initial_lr * 0.5 * (1 + math.cos(math.pi * epoch / epochs))
        
        return callbacks.LearningRateScheduler(cosine_schedule, verbose=1)
    
    elif schedule_type == 'step':
        # Step decay - reduce LR at specific epochs
        def step_schedule(epoch, lr):
            if epoch < 30:
                return initial_lr
            elif epoch < 60:
                return initial_lr * 0.1
            else:
                return initial_lr * 0.01
        
        return callbacks.LearningRateScheduler(step_schedule, verbose=1)
    
    else:
        raise ValueError("Schedule type must be 'plateau', 'exponential', 'cosine', or 'step'")

In [ ]:
# ---------------------------------------------------------------------------- #
#                            MODEL COMPILATION                                 #
# ---------------------------------------------------------------------------- #

def compile_model_with_combined_loss(model, learning_rate=1e-3, loss_alpha=0.5):
    """
    Compile model with combined loss and Adam optimizer
    
    Args:
        model: U-Net model to compile
        learning_rate: Initial learning rate for Adam
        loss_alpha: Weight factor for combined loss
    """
    
    # Create Adam optimizer
    optimizer = optimizers.Adam(
        learning_rate=learning_rate,
        beta_1=0.9,              # Exponential decay rate for 1st moment estimates
        beta_2=0.999,            # Exponential decay rate for 2nd moment estimates
        epsilon=1e-7,            # Small constant for numerical stability
        amsgrad=False            # Whether to apply AMSGrad variant
    )
    
    # Create combined loss function
    loss_function = combined_loss(alpha=loss_alpha)
    
    # Compile model
    model.compile(
        optimizer=optimizer,
        loss=loss_function,
        metrics=[
            'accuracy',                    # Pixel accuracy
            dice_coefficient,              # Dice coefficient metric
            tf.keras.metrics.BinaryIoU()   # IoU metric #type: ignore
        ]
    )
    
    print(f"Model compiled with:")
    print(f"  - Optimizer: Adam (lr={learning_rate})")
    print(f"  - Loss: Combined BCE + Dice (alpha={loss_alpha})")
    print(f"  - Metrics: Accuracy, Dice Coefficient, IoU")

In [ ]:
# ---------------------------------------------------------------------------- #
#                         TRAINING SETUP WITH CALLBACKS                        #
# ---------------------------------------------------------------------------- #

def setup_training_callbacks(model_name="unet_carvana", 
                            lr_schedule_type='plateau',
                            initial_lr=1e-3):
    """
    Setup all training callbacks including learning rate scheduling
    
    Args:
        model_name: Name for saving model files
        lr_schedule_type: Type of learning rate scheduler
        initial_lr: Initial learning rate
    
    Returns:
        List of callback objects
    """
    
    callbacks_list = []
    
    # 1. Model Checkpoint - Save best model
    checkpoint = callbacks.ModelCheckpoint(
        filepath=f'{model_name}_best.h5',
        monitor='val_binary_io_u',        # Monitor validation IoU
        mode='max',                       # Maximize IoU
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    )
    callbacks_list.append(checkpoint)
    
    # 2. Early Stopping - Stop training if no improvement
    early_stop = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,                      # Wait 15 epochs before stopping
        verbose=1,
        restore_best_weights=True         # Restore best weights when stopping
    )
    callbacks_list.append(early_stop)
    
    # 3. Learning Rate Scheduler
    lr_scheduler = create_learning_rate_scheduler(
        initial_lr=initial_lr,
        schedule_type=lr_schedule_type
    )
    callbacks_list.append(lr_scheduler)
    
    # 4. CSV Logger - Log training metrics
    csv_logger = callbacks.CSVLogger(
        filename=f'{model_name}_training_log.csv',
        append=True                       # Append to existing file if it exists
    )
    callbacks_list.append(csv_logger)
    
    # 5. TensorBoard (optional) - For visualization
    tensorboard = callbacks.TensorBoard(
        log_dir=f'./logs/{model_name}',
        histogram_freq=1,                 # Log weight histograms every epoch
        write_graph=True,
        write_images=True
    )
    callbacks_list.append(tensorboard)
    
    return callbacks_list

In [ ]:
compile_model_with_combined_loss(model, learning_rate=1e-3, loss_alpha=0.5)

# Setup callbacks for training
training_callbacks = setup_training_callbacks(
    model_name="unet_carvana_combined_loss",
    lr_schedule_type='plateau',  # Try 'plateau', 'exponential', 'cosine', or 'step'
    initial_lr=1e-3
)

print(f"\nTraining callbacks setup complete!")
print(f"Number of callbacks: {len(training_callbacks)}")
for i, callback in enumerate(training_callbacks):
    print(f"  {i+1}. {type(callback).__name__}")

## 7. Train the U-Net Model

### Step-by-Step Instructions:

1. **Pre-Training Preparation**:
   - Verify data generators are working correctly
   - Test model architecture with a small batch
   - Ensure all paths and dependencies are properly set
   - Clear any previous model states or cached data
   - Set up logging and monitoring systems

2. **Training Process Initiation**:
   - Start the training process using model.fit() or custom training loop
   - Monitor initial loss values to ensure proper initialization
   - Check that gradients are flowing properly (not too large or small)
   - Verify that data loading is efficient (no bottlenecks)

3. **Training Monitoring Strategy**:
   - **Real-time Metrics Tracking**:
     - Monitor training and validation loss curves
     - Track IoU and Dice coefficient progression
     - Watch for signs of overfitting (validation loss increasing)
     - Monitor learning rate changes from schedulers
   
   - **Visual Progress Monitoring**:
     - Periodically visualize predictions on validation samples
     - Compare predictions at different training stages
     - Create side-by-side comparisons of original, ground truth, and prediction
     - Save sample predictions at regular intervals

4. **Training Curve Analysis**:
   - **Healthy Training Signs**:
     - Both training and validation loss decreasing
     - Validation metrics improving steadily
     - No large spikes or instabilities
     - Learning rate reductions leading to continued improvement
   
   - **Overfitting Detection**:
     - Training loss continues to decrease while validation loss increases
     - Large gap between training and validation metrics
     - Validation metrics plateau or degrade
   
   - **Underfitting Indicators**:
     - Both training and validation loss plateau at high values
     - Model predictions look poor on validation samples
     - Metrics remain low despite adequate training time

5. **Training Optimization Strategies**:
   - **If Overfitting Occurs**:
     - Increase dropout rates
     - Add more data augmentation
     - Reduce model complexity
     - Implement early stopping
     - Use regularization techniques
   
   - **If Underfitting Occurs**:
     - Increase model capacity (more filters/layers)
     - Reduce regularization
     - Increase learning rate
     - Train for more epochs
     - Check data quality and preprocessing

6. **Progress Visualization**:
   - Create plots showing loss evolution over epochs
   - Plot IoU and Dice coefficient trends
   - Generate learning rate schedules visualization
   - Create confusion matrices for validation set
   - Show sample predictions at different training stages

7. **Checkpoint Management**:
   - Regularly save model checkpoints
   - Keep multiple backup checkpoints
   - Monitor disk space usage
   - Implement automatic cleanup of old checkpoints
   - Save best model based on validation metrics

8. **Training Completion**:
   - Allow training to complete or trigger early stopping
   - Load the best model checkpoint
   - Save final model weights and architecture
   - Document final training metrics
   - Create comprehensive training report

### Expected Training Timeline:
- **Initial epochs (1-10)**: Rapid loss decrease, basic shape learning
- **Middle epochs (10-30)**: Refinement of boundaries, metric improvement
- **Final epochs (30+)**: Fine-tuning, potential overfitting watch
- **Total time**: Several hours to days depending on hardware and dataset size

### Troubleshooting Common Issues:
- **NaN losses**: Check learning rate, data normalization, or loss function
- **Slow training**: Verify data loading efficiency, consider mixed precision
- **Memory issues**: Reduce batch size, optimize data pipeline
- **Poor convergence**: Adjust learning rate, check data quality

In [ ]:
# ---------------------------------------------------------------------------- #
#                         PREPARE TRAINING VARIABLES                           #
# ---------------------------------------------------------------------------- #

# Create aliases for data generators to match training function expectations
train_generator = train_gen
val_generator = val_gen

print("✅ Data generator aliases created:")
print(f"  - train_generator: {type(train_generator).__name__}")
print(f"  - val_generator: {type(val_generator).__name__}")
print(f"  - Training batches per epoch: {len(train_generator)}")
print(f"  - Validation batches per epoch: {len(val_generator)}")
print(f"  - Batch size: {train_generator.batch_size}")
print(f"  - Target size: {train_generator.target_size}")

# Verify generators work by testing a batch
print("\n🔍 Testing data generators...")
try:
    train_batch = next(iter(train_generator))
    val_batch = next(iter(val_generator))
    
    print(f"  ✅ Training batch shape: {train_batch[0].shape} (images), {train_batch[1].shape} (masks)")
    print(f"  ✅ Validation batch shape: {val_batch[0].shape} (images), {val_batch[1].shape} (masks)")
    print("  ✅ Data generators are working correctly!")
    
except Exception as e:
    print(f"  ❌ Error testing generators: {str(e)}")
    print("  Please check your data paths and generator configuration.")

print("\n🚀 Ready for training!")

In [ ]:
# ---------------------------------------------------------------------------- #
#                            TRAIN THE U-NET MODEL                             #
# ---------------------------------------------------------------------------- #

import time
import matplotlib.pyplot as plt
from datetime import datetime

def check_training_prerequisites():
    """
    Check if all required variables are defined before training
    """
    required_vars = {
        'model': 'U-Net model (from section 5)',
        'train_generator': 'Training data generator (from section 4)',
        'val_generator': 'Validation data generator (from section 4)',
        'training_callbacks': 'Training callbacks (from section 6)'
    }
    
    missing_vars = []
    for var_name, description in required_vars.items():
        if var_name not in globals():
            missing_vars.append(f"  ❌ {var_name}: {description}")
        else:
            print(f"  ✅ {var_name}: Found")
    
    if missing_vars:
        print("⚠️  MISSING VARIABLES - Please run these sections first:")
        for missing in missing_vars:
            print(missing)
        print("\nTo fix this:")
        print("1. Run all cells in order from the beginning")
        print("2. Make sure sections 1-6 are executed successfully")
        print("3. Then re-run this training cell")
        return False
    
    print("✅ All prerequisites found! Ready for training.")
    return True


def train_unet_model(model, train_generator, val_generator, 
                     epochs=50, steps_per_epoch=100, validation_steps=20,
                     callbacks_list=None, save_history=True):
    """
    Train the U-Net model with comprehensive monitoring
    
    Args:
        model: Compiled U-Net model
        train_generator: Training data generator
        val_generator: Validation data generator
        epochs: Number of training epochs
        steps_per_epoch: Steps per epoch for training
        validation_steps: Steps for validation
        callbacks_list: List of callbacks for training
        save_history: Whether to save training history
    
    Returns:
        Training history object
    """
    
    print("="*70)
    print("                    STARTING U-NET TRAINING")
    print("="*70)
    print(f"Training Configuration:")
    print(f"  - Epochs: {epochs}")
    print(f"  - Steps per epoch: {steps_per_epoch}")
    print(f"  - Validation steps: {validation_steps}")
    print(f"  - Callbacks: {len(callbacks_list) if callbacks_list else 0}")
    print(f"  - Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*70)
    
    # Record start time
    start_time = time.time()
    
    try:
        # Start training
        history = model.fit(
            train_generator,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            validation_data=val_generator,
            validation_steps=validation_steps,
            callbacks=callbacks_list,
            verbose=1,
            workers=4,              # Use multiple workers for data loading
            use_multiprocessing=True # Enable multiprocessing
        )
        
        # Calculate training time
        end_time = time.time()
        training_time = end_time - start_time
        
        print("\n" + "="*70)
        print("                    TRAINING COMPLETED")
        print("="*70)
        print(f"Total training time: {training_time/3600:.2f} hours")
        print(f"Average time per epoch: {training_time/epochs:.2f} seconds")
        print("="*70)
        
        # Save training history if requested
        if save_history:
            import pickle
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            history_filename = f'training_history_{timestamp}.pkl'
            
            with open(history_filename, 'wb') as f:
                pickle.dump(history.history, f)
            print(f"Training history saved to: {history_filename}")
        
        return history
        
    except Exception as e:
        print(f"\n⚠️  TRAINING ERROR: {str(e)}")
        print("Please check your configuration and try again.")
        raise


def visualize_training_progress(history, save_plots=True):
    """
    Create comprehensive visualizations of training progress
    
    Args:
        history: Training history object from model.fit()
        save_plots: Whether to save plots to files
    """
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('U-Net Training Progress', fontsize=16, fontweight='bold')
    
    # Plot 1: Loss curves
    axes[0, 0].plot(history.history['loss'], label='Training Loss', linewidth=2)
    axes[0, 0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    axes[0, 0].set_title('Model Loss', fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: IoU scores
    if 'binary_io_u' in history.history:
        axes[0, 1].plot(history.history['binary_io_u'], label='Training IoU', linewidth=2)
        axes[0, 1].plot(history.history['val_binary_io_u'], label='Validation IoU', linewidth=2)
        axes[0, 1].set_title('IoU Score', fontweight='bold')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('IoU')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Dice coefficient
    if 'dice_coefficient' in history.history:
        axes[1, 0].plot(history.history['dice_coefficient'], label='Training Dice', linewidth=2)
        axes[1, 0].plot(history.history['val_dice_coefficient'], label='Validation Dice', linewidth=2)
        axes[1, 0].set_title('Dice Coefficient', fontweight='bold')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Dice Score')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Accuracy
    if 'accuracy' in history.history:
        axes[1, 1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
        axes[1, 1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
        axes[1, 1].set_title('Pixel Accuracy', fontweight='bold')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Accuracy')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save plot if requested
    if save_plots:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        plot_filename = f'training_progress_{timestamp}.png'
        plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
        print(f"Training progress plot saved to: {plot_filename}")
    
    plt.show()


def monitor_training_predictions(model, val_generator, save_samples=True):
    """
    Generate and visualize sample predictions during training
    
    Args:
        model: Trained model
        val_generator: Validation data generator
        save_samples: Whether to save sample images
    """
    
    # Get a batch of validation data
    val_batch = next(val_generator)
    images, true_masks = val_batch
    
    # Generate predictions
    predictions = model.predict(images)
    
    # Select first few samples for visualization
    num_samples = min(4, len(images))
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4*num_samples))
    fig.suptitle('Training Progress - Sample Predictions', fontsize=16, fontweight='bold')
    
    for i in range(num_samples):
        # Original image
        axes[i, 0].imshow(images[i])
        axes[i, 0].set_title(f'Sample {i+1} - Original')
        axes[i, 0].axis('off')
        
        # True mask
        axes[i, 1].imshow(true_masks[i], cmap='gray')
        axes[i, 1].set_title(f'Sample {i+1} - True Mask')
        axes[i, 1].axis('off')
        
        # Predicted mask
        pred_mask = (predictions[i] > 0.5).astype(np.uint8)
        axes[i, 2].imshow(pred_mask, cmap='gray')
        axes[i, 2].set_title(f'Sample {i+1} - Prediction')
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    
    # Save samples if requested
    if save_samples:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        sample_filename = f'training_samples_{timestamp}.png'
        plt.savefig(sample_filename, dpi=300, bbox_inches='tight')
        print(f"Sample predictions saved to: {sample_filename}")
    
    plt.show()


# ---------------------------------------------------------------------------- #
#                         EXECUTE TRAINING PROCESS                             #
# ---------------------------------------------------------------------------- #

print("🔍 Checking training prerequisites...")

# Check if all required variables are available
if check_training_prerequisites():
    print("\n🚀 Starting U-Net Training Process...")
    
    try:
        # Train the model
        training_history = train_unet_model(
            model=model,
            train_generator=train_generator,
            val_generator=val_generator,
            epochs=50,                    # Adjust based on your needs
            steps_per_epoch=100,          # Adjust based on dataset size
            validation_steps=20,          # Adjust based on validation set size
            callbacks_list=training_callbacks,
            save_history=True
        )

        print("\n📊 Generating training visualizations...")

        # Visualize training progress
        visualize_training_progress(training_history, save_plots=True)

        print("\n🔍 Monitoring sample predictions...")

        # Monitor training predictions
        monitor_training_predictions(model, val_generator, save_samples=True)

        print("\n✅ Training process completed successfully!")
        print("\nNext steps:")
        print("  1. Review training curves for overfitting/underfitting")
        print("  2. Examine sample predictions for quality assessment")
        print("  3. Proceed to comprehensive model evaluation")
        print("  4. Consider hyperparameter tuning if needed")
        
    except Exception as e:
        print(f"\n❌ Training failed: {str(e)}")
        print("Please check the error message above and fix any issues.")

else:
    print("\n📝 To run training successfully:")
    print("1. Execute all notebook cells from the beginning (Ctrl+Shift+F10)")
    print("2. Or run cells 1-46 individually to set up all prerequisites")
    print("3. Then re-run this training cell")

In [ ]:
# ---------------------------------------------------------------------------- #
#                    RECREATE GENERATORS WITH FIXED CLASS                      #
# ---------------------------------------------------------------------------- #

print("🔧 Recreating data generators with tf.keras.utils.Sequence inheritance...")

# Recreate the generators using the updated class
train_gen_fixed, val_gen_fixed = create_data_generators(
    images_dir=images_dir, 
    masks_dir=masks_dir,
    batch_size=16,
    target_size=(512, 512),
    validation_split=0.2
)

# Update the aliases
train_generator = train_gen_fixed
val_generator = val_gen_fixed

print("✅ Updated generators created successfully!")
print(f"  - Generator type: {type(train_generator).__name__}")
print(f"  - Inherits from Sequence: {isinstance(train_generator, tf.keras.utils.Sequence)}")
print(f"  - Training batches: {len(train_generator)}")
print(f"  - Validation batches: {len(val_generator)}")

# Test the updated generators
print("\n🧪 Testing updated generators...")
try:
    train_batch = train_generator[0]  # Get first batch using Sequence interface
    val_batch = val_generator[0]
    
    print(f"  ✅ Training batch shape: {train_batch[0].shape} (images), {train_batch[1].shape} (masks)")
    print(f"  ✅ Validation batch shape: {val_batch[0].shape} (images), {val_batch[1].shape} (masks)")
    print("  ✅ Generators are now compatible with model.fit()!")
    
except Exception as e:
    print(f"  ❌ Error with updated generators: {str(e)}")

print("\n🚀 Generators are now ready for training with model.fit()!")

## 8. Model Evaluation and Metrics

### Step-by-Step Instructions:

1. **Load Best Model**:
   - Load the model checkpoint with the best validation performance
   - Verify model architecture is intact
   - Ensure proper weight loading without errors
   - Test model with a sample input to confirm functionality

2. **Comprehensive Metric Calculation**:
   
   **A. IoU (Intersection over Union)**:
   - Calculate for each image in validation/test set
   - Formula: IoU = (True Positives) / (True Positives + False Positives + False Negatives)
   - Compute mean IoU across all test images
   - Analyze IoU distribution and identify outliers
   
   **B. Dice Coefficient**:
   - Calculate pixel-wise Dice score for each prediction
   - Formula: Dice = 2 * (True Positives) / (2 * True Positives + False Positives + False Negatives)
   - Report mean and standard deviation across test set
   - Compare with IoU results for consistency
   
   **C. Pixel Accuracy**:
   - Calculate percentage of correctly classified pixels
   - Formula: Accuracy = (True Positives + True Negatives) / Total Pixels
   - Note limitations due to class imbalance
   
   **D. Precision and Recall**:
   - Precision = True Positives / (True Positives + False Positives)
   - Recall = True Positives / (True Positives + False Negatives)
   - Calculate F1-score: 2 * (Precision * Recall) / (Precision + Recall)

3. **Confusion Matrix Analysis**:
   - Create pixel-level confusion matrix
   - Visualize as heatmap for easy interpretation
   - Calculate additional metrics from confusion matrix:
     - Specificity (True Negative Rate)
     - False Positive Rate
     - False Negative Rate
   - Analyze class-wise performance

4. **Error Analysis**:
   - **Identify Failure Cases**:
     - Images with lowest IoU scores
     - Common failure patterns (shadows, reflections, similar colors)
     - Challenging scenarios (partial occlusion, complex backgrounds)
   
   - **Boundary Analysis**:
     - Evaluate performance at object boundaries
     - Measure boundary precision and recall
     - Identify systematic boundary errors
   
   - **Size-based Analysis**:
     - Performance on small vs. large objects
     - Correlation between object size and accuracy
     - Identify size-related biases

5. **Comparative Analysis**:
   - **Baseline Comparison**:
     - Compare against simple threshold-based methods
     - Benchmark against other segmentation algorithms
     - Document improvement over baseline approaches
   
   - **Ablation Studies**:
     - Impact of different loss functions
     - Effect of data augmentation strategies
     - Skip connection importance analysis
     - Architecture component contributions

6. **Statistical Significance Testing**:
   - Perform statistical tests on metric distributions
   - Calculate confidence intervals for mean metrics
   - Test for significant differences between model variants
   - Report statistical significance of improvements

7. **Visualization of Results**:
   - Create metric distribution histograms
   - Plot correlation between different metrics
   - Generate box plots for metric comparisons
   - Create scatter plots of metrics vs. image characteristics

8. **Performance Benchmarking**:
   - **Speed Analysis**:
     - Measure inference time per image
     - Analyze memory usage during inference
     - Compare with other model architectures
   
   - **Scalability Testing**:
     - Performance on different image sizes
     - Batch processing efficiency
     - GPU vs. CPU inference comparison

9. **Model Robustness Evaluation**:
   - Test on images with different characteristics:
     - Various lighting conditions
     - Different car colors and types
     - Diverse backgrounds
     - Different image qualities
   
   - **Generalization Assessment**:
     - Performance on held-out test set
     - Cross-validation results
     - Sensitivity to hyperparameter changes

### Expected Performance Benchmarks:
- **Good IoU**: > 0.85 for car segmentation
- **Excellent IoU**: > 0.90
- **Dice Coefficient**: Should be similar to IoU values
- **Pixel Accuracy**: Often > 0.95 but can be misleading
- **Processing Time**: < 100ms per image for real-time applications

### Documentation Requirements:
- Create comprehensive evaluation report
- Include visualizations of all key metrics
- Document methodology and assumptions
- Provide recommendations for model improvement

## 9. Prediction and Visualization

### Step-by-Step Instructions:

1. **Prediction Pipeline Setup**:
   - Load the trained model with best weights
   - Prepare prediction function with proper preprocessing
   - Ensure input images are processed consistently with training data
   - Set up batch prediction for efficiency
   - Handle different input image sizes if necessary

2. **Single Image Prediction Process**:
   - **Image Preprocessing**:
     - Resize image to model input size
     - Normalize pixel values to [0, 1] range
     - Add batch dimension for model input
     - Ensure correct data type (float32)
   
   - **Model Inference**:
     - Pass preprocessed image through trained model
     - Apply sigmoid activation to get probabilities
     - Convert probabilities to binary mask using threshold (typically 0.5)
     - Remove batch dimension from output
   
   - **Post-processing**:
     - Resize prediction back to original image size
     - Apply morphological operations for noise reduction
     - Smooth boundaries if necessary
     - Convert to appropriate format for visualization

3. **Batch Prediction Strategy**:
   - Process multiple images efficiently
   - Use appropriate batch sizes based on memory constraints
   - Implement progress tracking for large test sets
   - Handle memory management for large datasets
   - Save predictions systematically

4. **Visualization Techniques**:
   
   **A. Side-by-Side Comparison**:
   - Create three-panel displays: Original | Ground Truth | Prediction
   - Use consistent color schemes across all visualizations
   - Add titles and labels for clarity
   - Highlight differences between ground truth and prediction
   
   **B. Overlay Visualizations**:
   - Overlay predicted mask on original image with transparency
   - Use distinct colors for different regions (car vs. background)
   - Create contour overlays showing predicted boundaries
   - Show confidence maps with color coding
   
   **C. Error Visualization**:
   - Highlight false positive regions (predicted car, actually background)
   - Show false negative regions (missed car pixels)
   - Create difference maps showing prediction errors
   - Use color coding: green for correct, red for false positive, blue for false negative

5. **Comprehensive Result Gallery**:
   - **Best Predictions**: Show examples with highest IoU scores
   - **Challenging Cases**: Display difficult scenarios the model handles well
   - **Failure Cases**: Analyze and visualize poor predictions
   - **Edge Cases**: Show performance on unusual or difficult images
   - **Progressive Examples**: Show predictions at different training stages

6. **Interactive Visualizations**:
   - Create sliders to adjust prediction threshold
   - Interactive plots showing effect of different post-processing
   - Zoom functionality for detailed boundary analysis
   - Toggle between different visualization modes

7. **Quantitative Visualization**:
   - **Metric Overlays**: Show IoU and Dice scores on each image
   - **Confidence Maps**: Visualize model certainty across different regions
   - **Boundary Quality**: Highlight boundary precision
   - **Size Analysis**: Color-code predictions by object size

8. **Comparative Analysis Visualization**:
   - Compare with baseline methods (simple thresholding, traditional CV)
   - Show improvements over different model iterations
   - Visualize ensemble predictions if multiple models are used
   - Display performance across different image categories

9. **Export and Documentation**:
   - Save high-quality prediction images for reports
   - Create summary visualizations showing overall performance
   - Generate prediction videos for dynamic visualization
   - Export results in various formats (PNG, PDF, etc.)

10. **Quality Assessment Visualization**:
    - **Boundary Analysis**: Zoom into boundary regions
    - **Texture Analysis**: Show performance on different textures
    - **Lighting Analysis**: Performance under different lighting conditions
    - **Scale Analysis**: Results on different car sizes

### Visualization Best Practices:
- Use consistent color schemes throughout
- Provide clear legends and labels
- Ensure high resolution for detailed analysis
- Include quantitative metrics in visualizations
- Create both individual and summary visualizations
- Make visualizations accessible and interpretable

### Advanced Visualization Techniques:
- **Attention Visualization**: Show which parts of the image the model focuses on
- **Feature Map Visualization**: Display intermediate layer activations
- **Gradient Visualization**: Show important regions for predictions
- **Uncertainty Visualization**: Display model confidence levels

## 10. Save and Load Model

### Step-by-Step Instructions:

1. **Model Saving Strategies**:
   
   **A. Complete Model Saving**:
   - Save entire model including architecture, weights, and optimizer state
   - Use model.save() for TensorFlow/Keras models
   - Creates a single file containing everything needed for inference
   - Recommended for deployment and sharing
   - Format: .h5 or SavedModel format
   
   **B. Weights-Only Saving**:
   - Save only the trained weights using model.save_weights()
   - Requires separate architecture definition for loading
   - More storage efficient
   - Useful when architecture might change
   - Format: .h5 or checkpoint files

2. **Architecture Documentation**:
   - Save model architecture as JSON or YAML
   - Document model configuration parameters
   - Save hyperparameters used during training
   - Create model summary and layer information
   - Store preprocessing parameters and requirements

3. **Comprehensive Saving Process**:
   - **Best Model Checkpoint**: Save model with highest validation performance
   - **Final Model**: Save model at end of training
   - **Multiple Checkpoints**: Save models at different training stages
   - **Backup Strategy**: Create multiple copies for safety
   - **Version Control**: Include timestamp and version information

4. **Metadata and Configuration Saving**:
   - **Training Configuration**:
     - Learning rate schedules used
     - Data augmentation parameters
     - Batch size and epoch information
     - Loss function and optimizer settings
   
   - **Data Configuration**:
     - Input image dimensions
     - Normalization parameters
     - Class information and labels
     - Dataset statistics and splits
   
   - **Performance Metrics**:
     - Final evaluation scores
     - Training history and curves
     - Best achieved metrics
     - Computational requirements

5. **Model Loading Process**:
   
   **A. Complete Model Loading**:
   - Load entire model using tf.keras.models.load_model()
   - Verify model integrity after loading
   - Test with sample input to ensure functionality
   - Check that predictions match expected behavior
   
   **B. Architecture + Weights Loading**:
   - Reconstruct model architecture from saved configuration
   - Load weights using model.load_weights()
   - Verify architecture matches saved weights
   - Compile model with appropriate settings

6. **Model Verification After Loading**:
   - **Functionality Testing**:
     - Run prediction on test image
     - Compare results with expected outputs
     - Verify output dimensions and data types
     - Check prediction consistency
   
   - **Performance Verification**:
     - Re-evaluate on validation set
     - Confirm metrics match saved values
     - Test inference speed and memory usage
     - Validate preprocessing pipeline

7. **Deployment Preparation**:
   - **Model Optimization**:
     - Convert to optimized formats (TensorRT, ONNX)
     - Apply quantization if needed
     - Create lightweight versions for mobile deployment
     - Test optimized models for accuracy retention
   
   - **Inference Pipeline**:
     - Create complete inference script
     - Include all preprocessing steps
     - Add error handling and validation
     - Document input/output specifications

8. **Version Management**:
   - **Model Versioning**:
     - Use semantic versioning (v1.0.0, v1.1.0, etc.)
     - Track changes between versions
     - Maintain compatibility documentation
     - Create migration guides for version updates
   
   - **Experiment Tracking**:
     - Link models to specific experiments
     - Track hyperparameter changes
     - Maintain training logs and metrics
     - Document model lineage and improvements

9. **Storage and Organization**:
   - **File Structure**:
     - Organize models by experiment date and version
     - Create separate folders for different model variants
     - Include README files with model descriptions
     - Maintain consistent naming conventions
   
   - **Backup Strategy**:
     - Store models in multiple locations
     - Use cloud storage for important models
     - Create compressed archives for long-term storage
     - Implement automated backup procedures

10. **Sharing and Distribution**:
    - **Model Packaging**:
      - Create complete packages with dependencies
      - Include usage examples and documentation
      - Provide model cards with performance information
      - Package with sample data for testing
    
    - **Documentation**:
      - Create comprehensive user guides
      - Include API documentation
      - Provide performance benchmarks
      - Add troubleshooting information

### Best Practices:
- Always test loaded models before deployment
- Include checksums for model integrity verification
- Document all dependencies and requirements
- Create automated testing for model loading
- Maintain backwards compatibility when possible
- Use descriptive naming conventions for model files

### File Organization Example:
```
models/
├── unet_carvana_v1.0/
│   ├── model.h5                 # Complete saved model
│   ├── weights.h5              # Model weights only
│   ├── architecture.json       # Model architecture
│   ├── config.yaml            # Training configuration
│   ├── metrics.json           # Performance metrics
│   ├── preprocessing.py       # Preprocessing functions
│   └── README.md             # Model documentation
└── checkpoints/
    ├── epoch_010.h5
    ├── epoch_020.h5
    └── best_model.h5
```

## Conclusion and Next Steps

### Summary

This notebook has provided a comprehensive educational guide to implementing U-Net for image segmentation using the Carvana dataset. You have learned:

1. **U-Net Architecture**: Understanding the encoder-decoder structure with skip connections
2. **Data Handling**: Proper preprocessing, augmentation, and loading strategies
3. **Training Process**: Model compilation, training monitoring, and optimization
4. **Evaluation**: Comprehensive metrics and performance analysis
5. **Deployment**: Model saving, loading, and preparation for production use

### Key Takeaways

- **Skip Connections**: Critical for preserving spatial information in segmentation tasks
- **Data Augmentation**: Essential for robust model performance and generalization
- **Proper Evaluation**: Multiple metrics provide comprehensive performance assessment
- **Training Monitoring**: Early detection of overfitting and training issues is crucial
- **Systematic Approach**: Following structured methodology ensures reproducible results

### Potential Improvements and Extensions

1. **Architecture Enhancements**:
   - Implement Attention U-Net for better feature focusing
   - Try U-Net++ or UNet3+ for improved performance
   - Experiment with different encoder backbones (ResNet, EfficientNet)
   - Add pyramid pooling modules for multi-scale features

2. **Advanced Training Techniques**:
   - Implement focal loss for hard example mining
   - Use progressive resizing during training
   - Apply test-time augmentation for improved predictions
   - Experiment with different optimization strategies

3. **Data Enhancement**:
   - Implement more sophisticated augmentation techniques
   - Use synthetic data generation for dataset expansion
   - Apply domain adaptation techniques
   - Experiment with semi-supervised learning approaches

4. **Deployment Optimizations**:
   - Model quantization for mobile deployment
   - TensorRT optimization for GPU inference
   - Edge deployment using TensorFlow Lite
   - Real-time processing pipeline development

### Real-World Applications

The techniques learned in this notebook can be applied to various domains:

- **Medical Imaging**: Organ segmentation, tumor detection
- **Autonomous Vehicles**: Road segmentation, object detection
- **Agriculture**: Crop monitoring, disease detection
- **Satellite Imagery**: Land use classification, environmental monitoring
- **Manufacturing**: Quality control, defect detection

### Further Learning Resources

1. **Advanced Segmentation Architectures**: DeepLab, Mask R-CNN, SegFormer
2. **Medical Image Segmentation**: Specialized techniques and challenges
3. **Video Segmentation**: Temporal consistency and tracking
4. **3D Segmentation**: Volumetric data processing
5. **Weak Supervision**: Learning with limited annotations

### Final Notes

Remember that successful image segmentation projects require:
- High-quality, well-annotated data
- Careful hyperparameter tuning
- Proper validation strategies
- Comprehensive evaluation
- Systematic experimentation and documentation

The foundation provided in this notebook will serve as a solid starting point for tackling more complex segmentation challenges in your specific domain of interest.

In [ ]:
# ISSUE RESOLVED! ✅
# 
# PROBLEM: Data generator was failing with "No such file or directory" error
# ROOT CAUSE: File naming mismatch between images and masks
#   - Image files: 00087a6bd4dc_01.npy (no suffix)
#   - Mask files: 00087a6bd4dc_01_mask.npy (with "_mask" suffix)
#   - Generator was looking for mask files with same name as image files
#
# SOLUTION: Modified AlbumentationsDataGenerator class to:
#   1. Derive mask filenames from image filenames by adding "_mask" suffix
#   2. Verify all corresponding mask files exist during initialization
#   3. Updated create_data_generators function to use correct naming convention
#
# VERIFICATION: Data generator now works correctly!

print("🎉 DATA GENERATOR ISSUE SUCCESSFULLY RESOLVED!")
print("=" * 50)
print("✅ File naming mismatch fixed")
print("✅ AlbumentationsDataGenerator class updated") 
print("✅ Batch loading now works correctly")
print("✅ Ready for U-Net model training!")
print("=" * 50)

# Test final working generator
final_generator = AlbumentationsDataGenerator(
    images_dir="carvana-dataset/normalized_images_512x512",
    masks_dir="carvana-dataset/normalized_masks_512x512",
    batch_size=8,
    target_size=(512, 512),
    augmentation=None,
    shuffle=True
)

batch = final_generator[0]
X, y = batch
print(f"Final test - Batch shape: X{X.shape}, y{y.shape}")
print(f"Data ranges: X[{X.min():.3f}, {X.max():.3f}], y[{y.min():.3f}, {y.max():.3f}]")
print("✅ All systems ready for training!")